In [1]:
def read_log_file(file_name):
    parsed_log_lines = []
    metric_unit = None
    metric_name = None
    with open(file_name, "r") as log_file:
        for line in log_file.readlines():
            log_line = line.split()
            parsed_log_lines.append(create_dictionary_from_log_line(log_line))

            if not metric_name:
                metric_name = log_line[0]

            if not metric_unit:
                metric_unit = log_line[4]

    return parsed_log_lines, metric_unit, metric_name 


In [2]:
# read sample .log file 
fpath = '../data/MHEALTH_Dataset/mHealth_subject1.log'
with open(fpath, "r") as pf:
    i = 0
    for line in pf.readlines():
        print(f"L:{i} > {line}")
        i+=1
        break

L:0 > -9.8184	0.009971	0.29563	0.0041863	0.0041863	2.1849	-9.6967	0.63077	0.1039	-0.84053	-0.68762	-0.37	-0.36327	0.29963	-8.6499	-4.5781	0.18776	-0.44902	-1.0103	0.034483	-2.35	-1.6102	-0.030899	0



---

In [3]:
import numpy as np 
import pandas as pd 

In [ ]:
data_df = pd.read_csv('../data/MHEALTH_Dataset/mhealth_raw_data.csv')
data_df.head()

In [ ]:
for i, r in data_df.iterrows():
    print(r[:12].values, '>> ', r[12])
    break

[2.1849 -9.6967 0.6307699999999999 0.1039 -0.8405299999999999 -0.68762
 -8.6499 -4.5781 0.1877599999999999 -0.44902 -1.0103 0.034483] >>  0


In [ ]:
all_data = {"data": {}, "target": {}, 'collection': []}
prev_action = -1
starting = True
# action_seq = []
action_ID = 0
for i, s in data_df.iterrows():
    # if s[1] != "0":
    if (prev_action != int(s[12])):
        if not(starting):
            all_data['data'][action_ID] = np.array(action_seq)
            all_data['target'][action_ID] = prev_action
            action_ID+=1
        action_seq = []
    else:
        starting = False
    # data_seq = np.nan_to_num(np.array(s[3:]), nan=0).astype(np.float16)
    # data_seq[np.isnan(data_seq)] = 0
    data_seq = s[:12].values
    action_seq.append(data_seq)
    prev_action = int(s[12])
    all_data['collection'].append(data_seq)

In [ ]:
all_data['target']

{0: 0,
 1: 1,
 2: 0,
 3: 2,
 4: 0,
 5: 3,
 6: 0,
 7: 4,
 8: 0,
 9: 6,
 10: 0,
 11: 7,
 12: 0,
 13: 8,
 14: 0,
 15: 9,
 16: 0,
 17: 10,
 18: 0,
 19: 11,
 20: 0,
 21: 12,
 22: 0,
 23: 5,
 24: 0,
 25: 5,
 26: 0,
 27: 1,
 28: 0,
 29: 2,
 30: 0,
 31: 3,
 32: 0,
 33: 4,
 34: 0,
 35: 6,
 36: 0,
 37: 7,
 38: 0,
 39: 8,
 40: 0,
 41: 10,
 42: 0,
 43: 11,
 44: 0,
 45: 12,
 46: 0,
 47: 9,
 48: 0,
 49: 1,
 50: 0,
 51: 2,
 52: 0,
 53: 3,
 54: 0,
 55: 4,
 56: 0,
 57: 6,
 58: 0,
 59: 7,
 60: 0,
 61: 8,
 62: 0,
 63: 9,
 64: 0,
 65: 10,
 66: 0,
 67: 11,
 68: 0,
 69: 12,
 70: 0,
 71: 5,
 72: 0,
 73: 5,
 74: 0,
 75: 1,
 76: 0,
 77: 2,
 78: 0,
 79: 3,
 80: 0,
 81: 4,
 82: 0,
 83: 6,
 84: 0,
 85: 7,
 86: 0,
 87: 8,
 88: 0,
 89: 9,
 90: 0,
 91: 10,
 92: 0,
 93: 11,
 94: 0,
 95: 12,
 96: 0,
 97: 1,
 98: 0,
 99: 2,
 100: 0,
 101: 3,
 102: 0,
 103: 4,
 104: 0,
 105: 6,
 106: 0,
 107: 7,
 108: 0,
 109: 8,
 110: 0,
 111: 9,
 112: 0,
 113: 10,
 114: 0,
 115: 11,
 116: 0,
 117: 12,
 118: 0,
 119: 5,
 120: 0,
 121: 

---

In [5]:
import os
import random 
import numpy as np
import pandas as pd
# from scipy.signal import resample
from glob import glob 
import scipy.io 

from sklearn.preprocessing import MinMaxScaler

In [6]:
# build MHEALTH dataset data reader
class MHEALTHReader(object):
    def __init__(self, file_path):
        self.file_path = file_path
        self.readMH()

    def readMHFiles(self, data_df):
        data, labels, collection = [], [], []

        prev_action = -1
        starting = True
        # action_seq = []
        action_ID = 0

        for i, s in data_df.iterrows():
            if str(s[12]) != "0":
                if (prev_action != int(s[12])):
                    if not(starting):
                        data.append(np.array(action_seq))
                        labels.append(prev_action)
                        action_ID+=1
                    action_seq = []
                else:
                    starting = False

                data_seq = s[:12].values
                # data_seq[np.isnan(data_seq)] = 0
                action_seq.append(data_seq)
                prev_action = int(s[12])
                # print(prev_action)
                collection.append(data_seq)
        else: 
            if len(action_seq) > 1:
                data.append(np.array(action_seq))
                labels.append(prev_action)
        return np.asarray(data), np.asarray(labels, dtype=int), np.array(collection)

    def readMH(self):  
        label_map = [
            #(0, 'other'),
            (1, 'Standing still'),
            (2, 'Sitting and relaxing'),
            (3, 'Lying down'),
            (4, 'Walking'),
            (5, 'Climbing stairs'),
            (6, 'Waist bends forward'),
            (7, 'Frontal elevation of arms'), 
            (8, 'Knees bending (crouching)'),
            (9, 'Cycling'),
            (10, 'Jogging'),
            (11, 'Running'),
            (12, 'Jump front & back')
        ]
        labelToId = {x[0]: i for i, x in enumerate(label_map)}
        # print "label2id=",labelToId
        idToLabel = [x[1] for x in label_map]
        # print "id2label=",idToLabel
        cols = [0,1,2,3,4,5,6,7,8,9,10,11]
        self.cols = cols
        self.df = pd.read_csv(self.file_path)
        # print "cols",cols
        self.data, self.targets, self.all_data = self.readMHFiles(self.df)
        # self.data = self.data[:, :, cols]
        # print(self.data)
        # nan_perc = np.isnan(self.data).astype(int).mean()
        # print("null value percentage ", nan_perc)
        # f = lambda x: labelToId[x]
        self.targets = np.array([labelToId[i] for i in list(self.targets)])
        self.label_map = label_map
        self.idToLabel = idToLabel
        # return data, idToLabel

    def resample(self, signal, freq=10):
        step_size = int(100/freq)
        seq_len, _ = signal.shape 
        resample_indx = np.arange(0, seq_len, step_size)
        resampled_sig = signal[resample_indx, :]
        return resampled_sig

    def windowing(self, signal, window_len, overlap):
        seq_len = int(window_len*100) # 100Hz compensation 
        overlap_len = int(overlap*100) # 100Hz
        l, _ = signal.shape
        if l > seq_len:
            windowing_points = np.arange(start=0, stop=l-seq_len, step=seq_len-overlap_len, dtype=int)[:-1]

            windows = [signal[p:p+seq_len, :] for p in windowing_points]
        else:
            windows = []
        return windows

    def resampling(self, data, targets, window_size, window_overlap, resample_freq):
        assert len(data) == len(targets), "# action data & # action labels are not matching"
        all_data, all_ids, all_labels = [], [], []
        for i, d in enumerate(data):
            # print(">>>>>>>>>>>>>>>  ", np.isnan(d).mean())
            label = targets[i]
            windows = self.windowing(d, window_size, window_overlap)
            for w in windows:
                # print(np.isnan(w).mean(), label, i)
                resample_sig = self.resample(w, resample_freq)
                # print(np.isnan(resample_sig).mean(), label, i)
                all_data.append(resample_sig)
                all_ids.append(i+1)
                all_labels.append(label)

        return all_data, all_ids, all_labels

    def generate(self, unseen_classes, window_size=5.21, window_overlap=1, resample_freq=10, smoothing=False, normalize=False, seen_ratio=0.2, unseen_ratio=0.8):
        
        def smooth(x, window_len=11, window='hanning'):
            if x.ndim != 1:
                    raise Exception('smooth only accepts 1 dimension arrays.')
            if x.size < window_len:
                    raise Exception("Input vector needs to be bigger than window size.")
            if window_len<3:
                    return x
            if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
                    raise Exception("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")
            s=np.r_[2*x[0]-x[window_len-1::-1],x,2*x[-1]-x[-1:-window_len:-1]]
            if window == 'flat': #moving average
                    w=np.ones(window_len,'d')
            else:  
                    w=eval('np.'+window+'(window_len)')
            y=np.convolve(w/w.sum(),s,mode='same')
            return y[window_len:-window_len+1]

        # assert all([i in list(self.label_map.keys()) for i in unseen_classes]), "Unknown Class label!"
        seen_classes = [i for i in range(len(self.idToLabel)) if i not in unseen_classes]
        unseen_mask = np.in1d(self.targets, unseen_classes)

        # build seen dataset 
        seen_data = self.data[np.invert(unseen_mask)]
        seen_targets = self.targets[np.invert(unseen_mask)]

        # build unseen dataset
        unseen_data = self.data[unseen_mask]
        unseen_targets = self.targets[unseen_mask]

        # resampling seen and unseen datasets 
        seen_data, seen_ids, seen_targets = self.resampling(seen_data, seen_targets, window_size, window_overlap, resample_freq)
        unseen_data, unseen_ids, unseen_targets = self.resampling(unseen_data, unseen_targets, window_size, window_overlap, resample_freq)

        seen_data, seen_targets = np.array(seen_data), np.array(seen_targets)
        unseen_data, unseen_targets = np.array(unseen_data), np.array(unseen_targets)

        if normalize:
            a, b, nft = seen_data.shape 
            intm_sdata = seen_data.reshape((-1, nft))
            intm_udata = unseen_data.reshape((-1, nft))

            scaler = MinMaxScaler()
            norm_sdata = scaler.fit_transform(intm_sdata)
            norm_udata = scaler.transform(intm_udata)

            seen_data = norm_sdata.reshape(seen_data.shape)
            unseen_data = norm_udata.reshape(unseen_data.shape)

        if smoothing:
            seen_data = np.apply_along_axis(smooth, axis=1, arr=seen_data)
            unseen_data = np.apply_along_axis(smooth, axis=1, arr=unseen_data)
        # train-val split
        seen_index = list(range(len(seen_targets)))
        random.shuffle(seen_index)
        split_point = int((1-seen_ratio)*len(seen_index))
        fst_index, sec_index = seen_index[:split_point], seen_index[split_point:]
        # print(type(fst_index), type(sec_index), type(seen_data), type(seen_targets))
        X_seen_train, X_seen_val, y_seen_train, y_seen_val = seen_data[fst_index,:], seen_data[sec_index,:], seen_targets[fst_index], seen_targets[sec_index]
        

        data = {'train': {
                        'X': X_seen_train,
                        'y': y_seen_train
                        },
                'eval-seen':{
                        'X': X_seen_val,
                        'y': y_seen_val
                        },
                'test': {
                        'X': unseen_data,
                        'y': unseen_targets
                        },
                'seen_classes': seen_classes,
                'unseen_classes': unseen_classes
                }

        return data
        

In [7]:
dataReader = MHEALTHReader('../data/MHEALTH_Dataset/mhealth_raw_data.csv')
actionList = dataReader.idToLabel


C:\Users\deela\AppData\Local\Temp\ipykernel_17504\923409940.py:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(data), np.asarray(labels, dtype=int), np.array(collection)


In [8]:
data_dict = dataReader.generate(unseen_classes=[1, 3], seen_ratio=0.2, unseen_ratio=0.8, window_size=5.21, window_overlap=4.21, resample_freq=10)

In [9]:
dataReader.idToLabel

['Standing still',
 'Sitting and relaxing',
 'Lying down',
 'Walking',
 'Climbing stairs',
 'Waist bends forward',
 'Frontal elevation of arms',
 'Knees bending (crouching)',
 'Cycling',
 'Jogging',
 'Running',
 'Jump front & back']

In [10]:
# training dataset
train_X, train_y = data_dict['train']['X'], data_dict['train']['y']
print("number of training samples : ", len(train_y))
s = np.unique(train_y, return_counts=True)
print("per class count : ", dict(zip(s[0], s[1])))

number of training samples :  1819
per class count :  {0: 205, 2: 199, 4: 226, 5: 179, 6: 181, 7: 185, 8: 202, 9: 205, 10: 195, 11: 42}


In [11]:
# Seen Evaluation dataset
Seval_X, Seval_y = data_dict['eval-seen']['X'], data_dict['eval-seen']['y']
print("number of training samples : ", len(Seval_y))
s = np.unique(Seval_y, return_counts=True)
print("per class count : ", dict(zip(s[0], s[1])))

number of training samples :  455
per class count :  {0: 45, 2: 51, 4: 48, 5: 48, 6: 56, 7: 51, 8: 48, 9: 45, 10: 55, 11: 8}


In [12]:
# Unseen Eval dataset
test_X, test_y = data_dict['test']['X'], data_dict['test']['y']
print("number of training samples : ", len(test_y))
s = np.unique(test_y, return_counts=True)
print("per class count : ", dict(zip(s[0], s[1])))

number of training samples :  500
per class count :  {1: 250, 3: 250}


In [13]:
print("Total number of data points : ", len(test_y)+len(Seval_y)+len(train_y))
print("Total number of unseen data : ", len(test_y))
print("Total number of seen data : ", len(Seval_y)+len(train_y))

Total number of data points :  2774
Total number of unseen data :  500
Total number of seen data :  2274
